# **ETL MLOps**
 Se realizará un proceso que se enfoca en la preparación y limpieza de los datos para su uso con una API y análisis posteriores.

--------------------

# 1. Extracción de la base de datos
La base de datos fue brindada en el siguinte [link](https://drive.google.com/drive/folders/1mfUVyP3jS-UMdKHERknkQ4gaCRCO2e1v?usp=drive_link), la cuál fue descargada y guardada en un repositorio local. Se cuenta con 4 datasets que serán previamente analizados para continuar con la transformación de los necesarios.

In [329]:
#Importación de librerias 
import pandas as pd 
import numpy as np 

In [330]:
#Lectura de los datasets
#Movies
movies = pd.read_csv("Datasets\movies_dataset.csv")

#Créditos
creditos = pd.read_csv("Datasets\credits.csv")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11512\2483670269.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv("Datasets\movies_dataset.csv")


In [331]:
#Movies 
movies.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [332]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [333]:
#Creditos 
creditos.head(5)

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


---------------------------------------

# 2. TRANSFORMACIÓN DE LOS DATOS 
Se realizará las siguientes transformaciones:
- Eliminar las columnas que no serán utilizadas: video,imdb_id,adult,original_title,poster_path y homepage.
- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.
- Los valores nulos del campo release date deben eliminarse.
- De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.
- Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.
- Desanidar los datos que sean necesarios pensando en las consultas de API y modelamiento



**2.1 Eliminar las columnas que no serán utilizadas: video,imdb_id,adult,original_title,poster_path y homepage.**

In [334]:
#Definimos la lista de las columnas que eliminaremos
lista_eliminar = ['video','imdb_id','adult','original_title','poster_path','homepage']

In [335]:
#Elimanamos las columnas con la función drop
movies = movies.drop(columns= lista_eliminar)

In [336]:
#Revisamos 
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  object 
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

**2.2 Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.**

In [337]:
#Cambiamos el type de budget para poder realizar operaciones
#errors = 'coerce' forzamos la conversión de los valores no numéricos como NaN
movies['budget'] = pd.to_numeric(movies['budget'], errors='coerce')

In [338]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45463 non-null  float64
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45460 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

In [339]:
#Remplazamos null por 0 para hacer esto usamos fillna especificando los valores de las columnas. 
movies.fillna({'revenue':0, 'budget':0}, inplace=True)

In [341]:
#Revisamos
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4494 non-null   object 
 1   budget                 45466 non-null  float64
 2   genres                 45466 non-null  object 
 3   id                     45466 non-null  object 
 4   original_language      45455 non-null  object 
 5   overview               44512 non-null  object 
 6   popularity             45461 non-null  object 
 7   production_companies   45463 non-null  object 
 8   production_countries   45463 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45466 non-null  float64
 11  runtime                45203 non-null  float64
 12  spoken_languages       45460 non-null  object 
 13  status                 45379 non-null  object 
 14  tagline                20412 non-null  object 
 15  ti

**2.3 Los valores nulos del campo release date deben eliminarse.**

In [342]:
#Borramos los nulos de la columna especificando con subset
movies.dropna(subset=['release_date'], inplace=True)

In [343]:
#Revisamos
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45379 entries, 0 to 45465
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45379 non-null  float64
 2   genres                 45379 non-null  object 
 3   id                     45379 non-null  object 
 4   original_language      45368 non-null  object 
 5   overview               44438 non-null  object 
 6   popularity             45377 non-null  object 
 7   production_companies   45379 non-null  object 
 8   production_countries   45379 non-null  object 
 9   release_date           45379 non-null  object 
 10  revenue                45379 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

**2.4 De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.**

In [344]:
#Transformamos la columna de object a datetime
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')

In [345]:
#Extraemos el año en una nueva columna 
movies['release_year'] = movies['release_date'].dt.year.astype('Int64')

In [346]:
#Eliminamos la parte horaria de le fecha para que quede en el formato especificado
movies['release_date'] = movies['release_date'].dt.date

In [347]:
#En la elaboración habia datos que no corrian, comprobamos la que nos arrojaba error para continuar
print(movies.iloc[10395]['release_date'])
print(type(movies.iloc[10395]['release_date']))

2005-01-25
<class 'datetime.date'>


In [348]:
#Revisamos
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45379 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4491 non-null   object 
 1   budget                 45379 non-null  float64
 2   genres                 45379 non-null  object 
 3   id                     45379 non-null  object 
 4   original_language      45368 non-null  object 
 5   overview               44438 non-null  object 
 6   popularity             45377 non-null  object 
 7   production_companies   45379 non-null  object 
 8   production_countries   45379 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45379 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

In [349]:
#Debido a que por la transformación aparecieron valores nulos, se borran de nuevo.
movies.dropna(subset=['release_date'], inplace=True)

In [350]:
#Verificamos
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  float64
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  object 
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

**2.5 Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.**

In [351]:
#Para la división usaré 3 funciones que garanticen el codigo
#"divide" permite hacer la división, "replace" para cambiar los valores infinitos producto de los 0 y "fillna" para rellenar 0
movies['return'] = movies['revenue'].divide(movies['budget']).replace([np.inf, -np.inf],0).fillna(0)

In [352]:
#Revisamos
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  4488 non-null   object 
 1   budget                 45376 non-null  float64
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  object 
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

In [400]:
movies['return'].sample(10)

2655     0.000000
12899    0.000000
10134    3.118040
24042    0.000000
1531     0.849350
26455    0.000000
36189    0.000000
25716    0.000000
1948     0.000000
3045     0.092961
Name: return, dtype: float64

**2.6 Desanidar los datos que sean necesarios pensando en las consultas de API y modelamiento**

In [401]:
#Primero examiné las columnas anidadas para determinar su formato y los datos que son necesarios
movies.head(3)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.7129,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000


In [402]:
#belongs_to_collection,contiene diccionarios en su mayoria datos nulos, información importante: name
movies['belongs_to_collection'].sample(5).values

array([nan, nan,
       "{'id': 183256, 'name': 'Kiler Collection', 'poster_path': '/vEFCE3Dn9AndelMIfk9OsRUY5Of.jpg', 'backdrop_path': None}",
       nan, nan], dtype=object)

In [403]:
#genres, contiene listas de diccionarios, información importante: name
movies['genres'].sample(5).values

array(["[{'id': 27, 'name': 'Horror'}]",
       "[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 18, 'name': 'Drama'}]",
       '[]', "[{'id': 18, 'name': 'Drama'}]",
       "[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]"],
      dtype=object)

In [426]:
#production_companies, contiene listas de diccionarios, información importante: name
movies['production_companies'].sample(5).values

array(["[{'name': 'CJ Entertainment', 'id': 7036}]",
       "[{'name': 'Seasonal Film Corporation', 'id': 13433}, {'name': 'Shapiro-Glickenhaus Entertainment', 'id': 16636}]",
       '[]', '[]',
       '[{\'name\': "Institut National de l\'Audiovisuel (INA)", \'id\': 8171}]'],
      dtype=object)

In [430]:
#production_countries, contiene listas de diccionario, información importante: name
movies['production_countries'].sample(5).values

array(["[{'iso_3166_1': 'CA', 'name': 'Canada'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]",
       "[{'iso_3166_1': 'US', 'name': 'United States of America'}]",
       "[{'iso_3166_1': 'FR', 'name': 'France'}, {'iso_3166_1': 'DE', 'name': 'Germany'}, {'iso_3166_1': 'IT', 'name': 'Italy'}, {'iso_3166_1': 'US', 'name': 'United States of America'}]",
       '[]', "[{'iso_3166_1': 'US', 'name': 'United States of America'}]"],
      dtype=object)

In [432]:
#spoken_languages, contiene listas de diccionarios, información importante: name 
movies['spoken_languages'].sample(5).values

array(['[]', "[{'iso_639_1': 'en', 'name': 'English'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}]",
       "[{'iso_639_1': 'en', 'name': 'English'}]"], dtype=object)

**Conclusión:** 
- Todas tienen en común que son diccionarios donde el dato que necesito es la key "name". Se podría hacer una función que al llamarla acceda a esa key. 

 

- Se creara otra columna a partir de 'belongs_to_collection'. Para el resto creare una función que me permita acceder a estos valores name y coleccionarlos en string  

In [433]:
#Importamos una lib que nos permite manipular el código fuente de python para recorrer los diccionarios
import ast

**2.6.1 Columna Belongs_to_collection**

In [434]:
#Con la función lambda primero se condiciona el formato del dato y de ser nulo se rellena con None
movies['belongs_to_collection'] = movies['belongs_to_collection'].apply(lambda x: ast.literal_eval(x) 
                                                                        if pd.notnull(x) else 'None')

In [435]:
#Lambda tambien puede ser aplicado para extraer el valor de la key 'name'
movies['collection'] = movies['belongs_to_collection'].apply(lambda x: x.get('name', '')
                                                              if isinstance(x, dict) else 'None')

In [438]:
#Revisamos
movies.collection.sample(10)

22226                             None
30887    Hellboy (Animated) Collection
3395                              None
33119                             None
17117                             None
28358                             None
11010                Clerks Collection
45404                             None
25796                             None
6209                              None
Name: collection, dtype: object

In [439]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   belongs_to_collection  45376 non-null  object 
 1   budget                 45376 non-null  float64
 2   genres                 45376 non-null  object 
 3   id                     45376 non-null  object 
 4   original_language      45365 non-null  object 
 5   overview               44435 non-null  object 
 6   popularity             45376 non-null  object 
 7   production_companies   45376 non-null  object 
 8   production_countries   45376 non-null  object 
 9   release_date           45376 non-null  object 
 10  revenue                45376 non-null  float64
 11  runtime                45130 non-null  float64
 12  spoken_languages       45376 non-null  object 
 13  status                 45296 non-null  object 
 14  tagline                20398 non-null  object 
 15  ti

**2.6.2 Tratamiento de datos anidados en listas de diccionarios de movies**
- Se buscará aplicar una función que permita obtener una cadena string con los valores necesarios de las distintas columnas con listas de diccionarios. Previo a ello se hará uso de la biblioteca ast para poder tratar el dato como diccionario. 

- Definimos función para todos los casos de listas de diccionarios

In [440]:
def obtener_valores(lista_diccionario):
    """Función que recibe de argumento una lista de diccionarios y 
        y devuelve un String concatenado de estos valores """
    valores_name = []
    for diccionario in lista_diccionario:
        if isinstance(diccionario, dict) and "name" in diccionario:
            valores_name.append(diccionario["name"])
    return ", ".join(valores_name)

- **Generos**

In [441]:
#Definimos el tipo de dato de la columna genres como lista de diccionarios
movies['genres'] = movies['genres'].apply(ast.literal_eval)

In [442]:
#Creación de columna y aplicación de función 
movies['genre'] = movies['genres'].apply(obtener_valores)


In [447]:
movies['genre'].sample(10)

26399                                      Comedy
1499     Science Fiction, Horror, Drama, Thriller
28624     Action, Crime, Drama, Mystery, Thriller
44364                   Thriller, Comedy, Mystery
703                               Comedy, Foreign
27888                                Drama, Crime
15658                                 Documentary
45127                                       Drama
12838                     Crime, Horror, Thriller
19251                           Mystery, Thriller
Name: genre, dtype: object

In [448]:
#Tratamos los valores '' aplicando una función lambda condicional que cambia '' de existir por el valor None
movies['genre'] = movies['genre'].apply(lambda x: 'None'if x == '' else x )

In [462]:
movies['genre'].sample(3)

17576    Drama
9270     Drama
18503     None
Name: genre, dtype: object

- **production_companies**

In [463]:
#Definimos el tipo de dato de la columna production_companies como lista de diccionarios
movies['production_companies'] = movies['production_companies'].apply(ast.literal_eval)

In [464]:
#Creación de columna y aplicación de función 
movies['companies'] = movies['production_companies'].apply(obtener_valores)

In [473]:
movies['companies'].sample(3)

23423    Mad Dog Pictures, Spring Pictures
38547                                     
13956                                     
40456                 Walt Disney Pictures
1806                                      
12301           Merchant Ivory Productions
41893                   Pathé Distribution
26724                                     
13885               Caprino Filmcenter a/s
8702                          Latglen Ltd.
Name: companies, dtype: object

In [474]:
#Tratamos los valores '' aplicando una función lambda condicional que cambia '' de existir por el valor None
movies['companies'] = movies['companies'].apply(lambda x: 'None'if x == '' else x )

In [481]:
movies['companies'].sample(3)

19996    Angoa-Agicoa, Procirep, Canal+, CinéCinéma, Ré...
42175    DNA Films, TriStar Pictures, Cloud Eight Films...
16414                                                 None
Name: companies, dtype: object

- **production_countries**

In [482]:
#Definimos el tipo de dato de la columna production_countries como lista de diccionarios
movies['production_countries'] = movies['production_countries'].apply(ast.literal_eval)

In [483]:
#Creación de columna y aplicación de función 
movies['production_countrie'] = movies['production_countries'].apply(obtener_valores)

In [488]:
movies['production_countrie'].sample(3)

4248                   
33495    United Kingdom
41615            Israel
Name: production_countrie, dtype: object

In [489]:
#Tratamos los valores '' aplicando una función lambda condicional que cambia '' de existir por el valor None
movies['production_countrie'] = movies['production_countrie'].apply(lambda x: 'None'if x == '' else x )

In [490]:
movies['production_countrie'].sample(3)

24324                        None
26907    United States of America
840      United States of America
Name: production_countrie, dtype: object

- **spoken_languages**

In [491]:
#Definimos el tipo de dato de la columna spoken_languages como lista de diccionarios
movies['spoken_languages'] = movies['spoken_languages'].apply(ast.literal_eval)

In [492]:
#Creación de columna y aplicación de función 
movies['spoken_language'] = movies['spoken_languages'].apply(obtener_valores)

In [496]:
movies['spoken_language'].sample(3)

33835    한국어/조선말
18345           
3928     English
Name: spoken_language, dtype: object

In [497]:
#Tratamos los valores '' aplicando una función lambda condicional que cambia '' de existir por el valor None
movies['spoken_language'] = movies['spoken_language'].apply(lambda x: 'None'if x == '' else x )

In [504]:
movies['spoken_language'].sample(3)

19671                                                 None
27109                                              English
33704    English, Français, Deutsch, Italiano, 日本語, 한국어...
Name: spoken_language, dtype: object

**2.6.3 Desanidado de la columna crew en el dataset credits buscando el valor del director**
- Se analizara el dato a desanidar, se definira una función para que busque el nombre del director y se acoplara la columna al dataset movies. 

- Analisis de dataset

In [514]:
creditos.sample(3)

,cast,crew,id
5744,"[{'cast_id': 1, 'character': 'Jim Hawkins (voi...","[{'credit_id': '52fe44d1c3a36847f80abe45', 'de...",9016
43156,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5849ed3692514119af018478', 'de...",429450
18312,"[{'cast_id': 4, 'character': ""James 'Jim' Fisk...","[{'credit_id': '573f6953c3a3683bd6000631', 'de...",52359


In [506]:
creditos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB


In [508]:
creditos.cast.sample(1).values

array(["[{'cast_id': 2, 'character': 'Eddie', 'credit_id': '52fe449ac3a368484e02c1ed', 'gender': 2, 'id': 2228, 'name': 'Sean Penn', 'order': 0, 'profile_path': '/f2uy1zq0qmtLBpsr6N9uQ8FktOr.jpg'}, {'cast_id': 3, 'character': 'Mickey', 'credit_id': '52fe449ac3a368484e02c1f1', 'gender': 2, 'id': 1979, 'name': 'Kevin Spacey', 'order': 1, 'profile_path': '/cdowETe1PgXLjo72hDb7R7tyavf.jpg'}, {'cast_id': 4, 'character': 'Darlene', 'credit_id': '52fe449ac3a368484e02c1f5', 'gender': 1, 'id': 32, 'name': 'Robin Wright', 'order': 2, 'profile_path': '/tXfQTgcIEPP7gtVdJ44ZxZPhacn.jpg'}, {'cast_id': 5, 'character': 'Phil', 'credit_id': '52fe449ac3a368484e02c1f9', 'gender': 2, 'id': 9046, 'name': 'Chazz Palminteri', 'order': 3, 'profile_path': '/57fiwGZBohA7qytoXOUJtSNVksT.jpg'}, {'cast_id': 6, 'character': 'Artie', 'credit_id': '52fe449ac3a368484e02c1fd', 'gender': 2, 'id': 52865, 'name': 'Garry Shandling', 'order': 4, 'profile_path': '/rMSod5msflCZBZ9LfJu8n4zYkuP.jpg'}, {'cast_id': 7, 'character'

In [511]:
#El valor buscado se encuentra en la columna crew
creditos.crew.sample(1).values

array(["[{'credit_id': '52fe449e9251416c91016fdd', 'department': 'Sound', 'gender': 2, 'id': 636, 'job': 'Original Music Composer', 'name': 'Joe Hisaishi', 'profile_path': '/9U16jEYcoCyXGBpHrSQ7pZGdHfd.jpg'}, {'credit_id': '52fe449e9251416c91016fd1', 'department': 'Writing', 'gender': 2, 'id': 25622, 'job': 'Screenplay', 'name': 'Yoshikazu Yasuhiko', 'profile_path': '/7004TpfVfdyrARdWlSFMbA1aX0w.jpg'}, {'credit_id': '52fe449e9251416c91016fcb', 'department': 'Directing', 'gender': 2, 'id': 25622, 'job': 'Director', 'name': 'Yoshikazu Yasuhiko', 'profile_path': '/7004TpfVfdyrARdWlSFMbA1aX0w.jpg'}, {'credit_id': '52fe449e9251416c91016fd7', 'department': 'Writing', 'gender': 0, 'id': 555172, 'job': 'Screenplay', 'name': 'Yuichi Sasamoto', 'profile_path': None}]"],
      dtype=object)

El nombre del director/a se encuentra anidado en un diccionario dentro de datos tipo lista de diccionario que es especifico para el departamento de dirección. Es por ello que se definirá una función que acceda a esta lista, busque el diccionario y extraiga el valor de la key name.

- **Definición de función**

En busca de optimizar el código se usara la función next para que cuando la iteración encuentre el valor deseado esta concluya.

In [569]:
def obtener_director(crew_list):
    """ Función que recibe de argumento una lista de diccionarios, 
        busca el diccionario que contenga el departamento un valor buscado
        y lo retorna."""
    director_dict = next((member for member in crew_list if isinstance(member, dict) 
                          and member.get('department') == 'Directing'), None)
    return director_dict.get('name') if director_dict else None

- **Aplicación de función**

In [570]:
#Definimos el tipo de dato de la columna crew como lista de diccionarios con ast
creditos['crew'] = creditos['crew'].apply(ast.literal_eval)

In [571]:
#Aplicamos función y creamos la nueva columnas con los nombres de los directores
creditos['director_name'] = creditos['crew'].apply(obtener_director)

In [572]:
#Visualizamos
creditos.sample(10)

,cast,crew,id,director_name
14808,"[{'cast_id': 1, 'character': 'Louise', 'credit...","[{'credit_id': '52fe4571c3a368484e05b591', 'de...",27989,Cheryl Hines
24324,"[{'cast_id': 1001, 'character': '(voice)', 'cr...","[{'credit_id': '52fe4859c3a36847f81630e7', 'de...",53222,Jérémy Clapin
37969,"[{'cast_id': 6, 'character': 'Iris', 'credit_i...","[{'credit_id': '52fe4b75c3a368484e189e63', 'de...",131334,Fiorella Infascelli
31490,"[{'cast_id': 0, 'character': 'Detective Ellen ...",[],338234,None
17833,"[{'cast_id': 2, 'character': 'Sam Bisbee', 'cr...","[{'credit_id': '52fe4b7dc3a368484e18b6e1', 'de...",131732,Erle C. Kenton
16254,"[{'cast_id': 5, 'character': 'Himself', 'credi...","[{'credit_id': '52fe46069251416c910468cb', 'de...",36738,Dana Heinz Perry
4222,"[{'cast_id': 12, 'character': 'Narrator (voice...","[{'credit_id': '52fe44d29251416c9101e095', 'de...",32458,Costa Botes
7496,"[{'cast_id': 1, 'character': 'Takuma Tsurugi',...","[{'credit_id': '52fe45a5c3a36847f80d27a5', 'de...",40812,Shigehiro Ozawa
45384,"[{'cast_id': 1, 'character': 'Shyamlal Chatter...","[{'credit_id': '58877d529251415ff70053e7', 'de...",408624,Konkona Sen Sharma
40224,"[{'cast_id': 1, 'character': 'Pablo', 'credit_...","[{'credit_id': '54acaaecc3a36822b700524d', 'de...",316074,Claudia Lorenz


**2.7 Eliminación de columnas anidadas**

2.7.1 Dataset Movies

In [574]:
#Definimos la lista de columnas para borrar
lista_anidadas = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages']  
#Borramos
movies = movies.drop(columns= lista_anidadas)

In [575]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   budget               45376 non-null  float64
 1   id                   45376 non-null  object 
 2   original_language    45365 non-null  object 
 3   overview             44435 non-null  object 
 4   popularity           45376 non-null  object 
 5   release_date         45376 non-null  object 
 6   revenue              45376 non-null  float64
 7   runtime              45130 non-null  float64
 8   status               45296 non-null  object 
 9   tagline              20398 non-null  object 
 10  title                45376 non-null  object 
 11  vote_average         45376 non-null  float64
 12  vote_count           45376 non-null  float64
 13  release_year         45376 non-null  Int64  
 14  return               45376 non-null  float64
 15  collection           45376 non-null 

2.7.2 Dataset creditos

In [582]:
#Definimos la lista de columnas para borrar
lista_credits = ['cast', 'crew']  
#Borramos
creditos = creditos.drop(columns= lista_credits)

In [623]:
creditos.head(5)

,id,director_name
0,862,John Lasseter
1,8844,Joe Johnston
2,15602,Howard Deutch
3,31357,Forest Whitaker
4,11862,Charles Shyer


In [584]:
creditos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             45476 non-null  int64 
 1   director_name  44590 non-null  object
dtypes: int64(1), object(1)
memory usage: 710.7+ KB


--------------------------------

# 3. Load de la base de datos

**3.1 Unión de la base de datos**

In [585]:
#Transformamos del ID de Movies a tipo INT
movies['id'] = movies['id'].astype('int64')

In [586]:
#Unimos los datos con merge
movies_vf = pd.merge(movies, creditos, on = 'id')

In [589]:
movies_vf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45451 entries, 0 to 45450
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   budget               45451 non-null  float64
 1   id                   45451 non-null  int64  
 2   original_language    45440 non-null  object 
 3   overview             44510 non-null  object 
 4   popularity           45451 non-null  object 
 5   release_date         45451 non-null  object 
 6   revenue              45451 non-null  float64
 7   runtime              45205 non-null  float64
 8   status               45371 non-null  object 
 9   tagline              20425 non-null  object 
 10  title                45451 non-null  object 
 11  vote_average         45451 non-null  float64
 12  vote_count           45451 non-null  float64
 13  release_year         45451 non-null  Int64  
 14  return               45451 non-null  float64
 15  collection           45451 non-null 

**3.2 Ordenado de la base de datos**

In [592]:
#Definimos lista de orden 
orden = ['id','title','genre','collection','original_language','spoken_language','runtime',
         'release_date','release_year','director_name','companies','production_countrie',
         'status','overview','popularity','vote_average','vote_count','budget','revenue','return','tagline']

In [593]:
#Ordenamos
movies_vf=movies_vf[orden]

In [601]:
movies_vf.sample()

,id,title,genre,collection,original_language,spoken_language,runtime,release_date,release_year,director_name,...,production_countrie,status,overview,popularity,vote_average,vote_count,budget,revenue,return,tagline
25602,120605,The Punisher: Dirty Laundry,"Action, Crime, Thriller",None,en,English,10.0,2012-07-16,2012,Phil Joanou,...,United States of America,Released,"Frank Castle is staying in a bad neighborhood,...",11.309444,7.3,144.0,0.0,0.0,0.0,What's the difference between justice and puni...


**3.3 Exportamos el dataset**

In [603]:
movies_vf.to_csv('movies_final.csv', index=False)

In [604]:
df = pd.read_csv('movies_final.csv')

In [606]:
df.sample(5)

,id,title,genre,collection,original_language,spoken_language,runtime,release_date,release_year,director_name,...,production_countrie,status,overview,popularity,vote_average,vote_count,budget,revenue,return,tagline
38891,147307,Multiple SIDosis,None,None,en,None,10.0,1970-12-31,1970,Sid Laverents,...,None,Released,Short film utilizing quick cuts and multiple a...,0.049417,5.0,1.0,0.0,0.0,0.000000,Short film of a one-man-band musical performan...
43450,30592,The Jacksons: An American Dream,"Drama, Music",None,en,English,240.0,1992-11-15,1992,Karen Arthur,...,United States of America,Released,The Jacksons are your average working-class fa...,1.717329,4.9,20.0,0.0,0.0,0.000000,NaN
16273,10140,The Chronicles of Narnia: The Voyage of the Da...,"Adventure, Family, Fantasy",The Chronicles of Narnia Collection,en,English,113.0,2010-08-13,2010,Michael Apted,...,United States of America,Released,"This time around Edmund and Lucy Pevensie, alo...",12.383398,6.2,1555.0,155000000.0,415686217.0,2.681847,Return to magic. Return to hope. Return to Nar...
32849,108586,A Bunny's Tale,None,None,en,None,104.0,1985-02-25,1985,NaN,...,None,Released,A dramatization of Gloria Steinem's undercover...,0.049482,5.0,1.0,0.0,0.0,0.000000,NaN
9240,844,2046,"Fantasy, Drama, Science Fiction, Romance",Wong Kar-Wai's Informal Collection,zh,"广州话 / 廣州話, 日本語, 普通话",129.0,2004-05-20,2004,Wong Kar-wai,...,"Germany, Hong Kong, China, France, Italy",Released,2046 is the sequel to Wong Kar-Wais’ successfu...,7.157169,6.9,208.0,12000000.0,19271312.0,1.605943,Are you still in the mood for love?


------------------------------------